# Playground Notebook For Quantizing VLP Models

## Initialize the Distributed Backend

In [1]:
import os
import torch
import pytorch_lightning as pl
import configs
import copy
from sensitivity_utils import get_quantization_config, print_size_of_model, SmallMTDataModuleMETER, SmallMTDataModuleVILT
# from vilt.datamodules.multitask_datamodule import MTDataModule as MTDataModuleVILT
# from meter.datamodules.multitask_datamodule import MTDataModule as MTDataModuleMeter

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import entropy



# Limit the number of CPUs
os.environ["OMP_NUM_THREADS"] = "8"  # Set this to the number of CPUs you want to use
os.environ["MKL_NUM_THREADS"] = "8"  # Set this to the number of CPUs you want to use

/data-4/users/mileriso/envs/.dev/lib/python3.10/site-packages/torch_xla/__init__.py:253: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(
2025-02-07 11:25:40.736683: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738923940.753245 2446998 cuda_dnn.cc:8498] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738923940.758352 2446998 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
import warnings

# Suppress specific warnings
warnings.filterwarnings("ignore", category=FutureWarning, message="promote has been superseded by promote_options='default'")
warnings.filterwarnings("ignore", category=FutureWarning, message="Importing from timm.models.helpers is deprecated")
warnings.filterwarnings("ignore", category=FutureWarning, message="Importing from timm.models.layers is deprecated")
warnings.filterwarnings("ignore", category=FutureWarning, message="Importing from timm.models.registry is deprecated")
warnings.filterwarnings("ignore", category=UserWarning, message="Overwriting vit_small_patch16_224 in registry")
warnings.filterwarnings("ignore", category=UserWarning, message="Overwriting vit_base_patch16_224 in registry")
warnings.filterwarnings("ignore", category=UserWarning, message="Overwriting vit_base_patch32_224 in registry")
warnings.filterwarnings("ignore", category=UserWarning, message="Overwriting vit_base_patch16_384 in registry")
warnings.filterwarnings("ignore", category=UserWarning, message="Overwriting vit_base_patch32_384 in registry")
warnings.filterwarnings("ignore", category=UserWarning, message="Overwriting vit_large_patch16_224 in registry")
warnings.filterwarnings("ignore", category=UserWarning, message="Overwriting vit_large_patch32_224 in registry")
warnings.filterwarnings("ignore", category=UserWarning, message="Overwriting vit_large_patch16_384 in registry")
warnings.filterwarnings("ignore", category=UserWarning, message="Overwriting vit_large_patch32_384 in registry")
warnings.filterwarnings("ignore", category=UserWarning, message="Overwriting vit_base_patch16_224_in21k in registry")
warnings.filterwarnings("ignore", category=UserWarning, message="Overwriting vit_base_patch32_224_in21k in registry")
warnings.filterwarnings("ignore", category=UserWarning, message="Overwriting vit_large_patch16_224_in21k in registry")
warnings.filterwarnings("ignore", category=UserWarning, message="Overwriting vit_large_patch32_224_in21k in registry")
warnings.filterwarnings("ignore", category=UserWarning, message="Overwriting vit_huge_patch14_224_in21k in registry")
warnings.filterwarnings("ignore", category=UserWarning, message="Overwriting vit_base_resnet50_224_in21k in registry")
warnings.filterwarnings("ignore", category=UserWarning, message="Overwriting vit_base_resnet50_384 in registry")
warnings.filterwarnings("ignore", category=UserWarning, message="Overwriting vit_small_resnet26d_224 in registry")
warnings.filterwarnings("ignore", category=UserWarning, message="Overwriting vit_base_resnet26d_224 in registry")
warnings.filterwarnings("ignore", category=UserWarning, message="Overwriting vit_base_resnet50d_224 in registry")
warnings.filterwarnings("ignore", category=FutureWarning, message="You are using `torch.load` with `weights_only=False`")

### Define the Configuration to Initialize the Datamodule and Model

In [3]:
# Set the configuration
_config = configs.meter_config_nlvr2
_config["batch_size"] = 64
_config["per_gpu_batchsize"] = 64


pl.seed_everything(_config["seed"])

Seed set to 0


0

## Initialize the Datamodule

Select the configuration and initialize the test and full datamodule

In [4]:
# ==========================================
# ========= Create full datamodule =========
# ==========================================
if "meter" in _config["model"]:
    infer_dm = SmallMTDataModuleMETER(_config, dist=False, num_samples=5, start_idx=0)
    infer_dm.setup("test", is_random=True)
    infer_dataloader = infer_dm.test_dataloader()

elif "vilt" in _config["model"]:
    infer_dm = SmallMTDataModuleVILT(_config, dist=False, num_samples=5, start_idx=0)
    infer_dm.setup("test", is_random=True)
    infer_dataloader = infer_dm.test_dataloader()

else:
    raise ValueError("Model not supported: ", _config["model"])

print(f"Batch size: {_config['batch_size']}")

Loaded names: ['nlvr2_vlue_test']
Loaded names: ['nlvr2_vlue_test']
Loaded names: ['nlvr2_vlue_test']
Batch size: 64


## Initialize The Model

In [5]:
from vilt.modules import ViLTransformerSS
from meter.modules import METERTransformerSS

if _config["model"] == "vilt":
    model = ViLTransformerSS(_config)
    print("Initialized ViLT model")

elif _config["model"] == "meter":
    model = METERTransformerSS(_config)
    print("Initialized METER model")

else:
    raise ValueError("Model not supported: ", _config["model"])

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Initialized METER model


In [6]:
input_ = next(iter(infer_dataloader))
print(input_.keys())
# print(input_["answers"])
print(f"Number of batches: {len(infer_dataloader)}")
print(f"Smaples in a batch: {len(input_['answers'])}")

dict_keys(['table_name', 'answers', 'text', 'image_1', 'image_0', 'text_ids', 'text_labels', 'text_ids_mlm', 'text_labels_mlm', 'text_masks'])
Number of batches: 1
Smaples in a batch: 5


# Quantize the Model

In [ ]:

quantization_config, embedding_config = get_quantization_config(2)

layer_to_quantize = "text_transformer.encoder.layer"
# layer_to_quantize = "transformer.blocks"


model_dynamic = copy.deepcopy(model)
torch.quantization.quantize_dynamic(
    model_dynamic, {layer_to_quantize: quantization_config}, inplace=True
)

# print(model_dynamic)
print("Size after quantization:")
print_size_of_model(model_dynamic)

Full Precision Model:
Size of the model (MB): 1296.258138
Size after quantization:
Size of the model (MB): 1041.51393


In [8]:
# def get_module_by_path(model, path):
#     """
#     Access a module in the model by specifying the path as a string.
    
#     Args:
#         model (nn.Module): The PyTorch model.
#         path (str): The path to the module, e.g., "transformer.blocks[0].attn.qkv".
    
#     Returns:
#         nn.Module: The module at the specified path.
#     """
#     parts = path.split('.')
#     current_module = model
    
#     for part in parts:
#         if '[' in part and ']' in part:
#             # Handle list indexing, e.g., "blocks[0]"
#             module_name, index = part.split('[')
#             index = int(index[:-1])  # Remove the closing bracket and convert to int
#             current_module = getattr(current_module, module_name)[index]
#         else:
#             # Handle regular attribute access
#             current_module = getattr(current_module, part)
    
#     return current_module

# # Example usage
# module_path = "transformer.blocks[0].mlp.fc2"
# qkv_module = get_module_by_path(model, module_path)
# print(qkv_module)

# Quant vs FP32 Plots

In [9]:
# import matplotlib.pyplot as plt
# import seaborn as sns

# def plot_weight_distributions(model, model_quant, block):
#     """Plot weight distributions for fc1 and fc2 layers."""
#     # Full-precision weights
#     fc1_full = model.transformer.blocks[block].mlp.fc1.weight.detach().flatten().cpu().numpy()
#     fc2_full = model.transformer.blocks[block].mlp.fc2.weight.detach().flatten().cpu().numpy()
    
#     # Quantized weights
#     fc1_quant = model_quant.transformer.blocks[block].mlp.fc1.weight().dequantize().detach().flatten().cpu().numpy()
#     fc2_quant = model_quant.transformer.blocks[block].mlp.fc2.weight().dequantize().detach().flatten().cpu().numpy()
    
#     # Plot fc1
#     plt.figure(figsize=(12, 5))
#     plt.subplot(1, 2, 1)
#     # sns.kdeplot(fc1_full, label='Full Precision', color='blue')
#     # sns.kdeplot(fc1_quant, label='Quantized', color='red')
#     plt.hist(fc1_full, bins=10, alpha=0.5, label='Full Precision', color='blue')
#     plt.hist(fc1_quant, bins=10, alpha=0.5, label='Quantized', color='red')
#     plt.title(f'Block {block} MLP.fc1 Weight Distribution')
#     plt.xlabel('Weight Value')
#     plt.legend()

#     # plt.xlim(-2, 2)
#     # plt.ylim(0,10)
    
    
#     # Plot fc2
#     plt.subplot(1, 2, 2)
#     # sns.kdeplot(fc2_full, label='Full Precision', color='blue')
#     # sns.kdeplot(fc2_quant, label='Quantized', color='red')
#     plt.hist(fc2_full, bins=10, alpha=0.5, label='Full Precision', color='blue')
#     plt.hist(fc2_quant, bins=10, alpha=0.5, label='Quantized', color='red')
#     plt.title(f'Block {block} MLP.fc2 Weight Distribution')
#     plt.xlabel('Weight Value')
#     plt.legend()

#     # plt.xlim(-2, 2)
#     # plt.ylim(0,10)
    
    
#     plt.tight_layout()
#     plt.show()

In [10]:
import math
import torch.nn.functional as F

def get_attn_outs(model, batch, block):
    """Get attention outputs for a specific block."""
    co_embeds, co_masks = model.get_co_embeds(batch, block)
    
    return co_embeds, co_masks

def weighted_variance_change(full_var, quant_var):
    """Calculate weighted variance change."""
    percent_change = ((quant_var - full_var) / full_var) * 100
    return percent_change * math.log(full_var)

def analyze_mlp_activations(model, model_quant, batch, block):
    """Compare activations of MLP layers (fc1 and fc2) for a specific block."""

    if _config["model"] == "vilt":
        x, co_masks = model.get_co_embeds(batch, block)
        x_quant, co_masks_quant = model_quant.get_co_embeds(batch, block)
        
        with torch.no_grad():
            # Full-precision activations
            x_fc1_full = model.transformer.blocks[block].mlp.fc1(x)
            x_fc2_full = model.transformer.blocks[block].mlp.fc2(x_fc1_full)
            
            # Quantized activations
            x_fc1_quant = model_quant.transformer.blocks[block].mlp.fc1(x_quant)
            x_fc2_quant = model_quant.transformer.blocks[block].mlp.fc2(x_fc1_quant)
    
    elif _config["model"] == "meter":
        x_fc1_full, x_fc2_full, final_outputs = model.get_mlp_outputs(batch, block)

        x_fc1_quant, x_fc2_quant, final_outputs_quant = model_quant.get_mlp_outputs(batch, block)
        
        
    
    # Compute metrics for fc1
    mse_fc1 = F.mse_loss(x_fc1_full, x_fc1_quant).item()
    cosine_fc1 = F.cosine_similarity(x_fc1_full.flatten(), x_fc1_quant.flatten(), dim=0).item()
    
    # Compute metrics for fc2
    mse_fc2 = F.mse_loss(x_fc2_full, x_fc2_quant).item()
    cosine_fc2 = F.cosine_similarity(x_fc2_full.flatten(), x_fc2_quant.flatten(), dim=0).item()
    
    # Activation statistics (mean and variance)
    fc1_full_mean = x_fc1_full.mean().item()
    fc1_full_var = x_fc1_full.var().item()
    fc1_quant_mean = x_fc1_quant.mean().item()
    fc1_quant_var = x_fc1_quant.var().item()
    
    fc2_full_mean = x_fc2_full.mean().item()
    fc2_full_var = x_fc2_full.var().item()
    fc2_quant_mean = x_fc2_quant.mean().item()
    fc2_quant_var = x_fc2_quant.var().item()
    
    # Percent changes in means and variances
    def percent_change(full, quant):
        return ((quant - full) / full) * 100
    
    fc1_mean_change = percent_change(fc1_full_mean, fc1_quant_mean)
    fc1_var_change = percent_change(fc1_full_var, fc1_quant_var)
    fc2_mean_change = percent_change(fc2_full_mean, fc2_quant_mean)
    fc2_var_change = percent_change(fc2_full_var, fc2_quant_var)
    
    # Weighted variance change
    fc1_var_weighted_change = weighted_variance_change(fc1_full_var, fc1_quant_var)
    fc2_var_weighted_change = weighted_variance_change(fc2_full_var, fc2_quant_var)
    
    # Return results
    return {
        'mse_fc1': mse_fc1,
        'cosine_fc1': cosine_fc1,
        'mse_fc2': mse_fc2,
        'cosine_fc2': cosine_fc2,
        'stats': {
            'fc1_full_mean': fc1_full_mean,
            'fc1_full_var': fc1_full_var,
            'fc1_quant_mean': fc1_quant_mean,
            'fc1_quant_var': fc1_quant_var,
            'fc2_full_mean': fc2_full_mean,
            'fc2_full_var': fc2_full_var,
            'fc2_quant_mean': fc2_quant_mean,
            'fc2_quant_var': fc2_quant_var,
            'fc1_mean_change': fc1_mean_change,
            'fc1_var_change': fc1_var_change,
            'fc2_mean_change': fc2_mean_change,
            'fc2_var_change': fc2_var_change,
            'fc1_var_weighted_change': fc1_var_weighted_change,
            'fc2_var_weighted_change': fc2_var_weighted_change,
        }
    }

In [11]:
# from sklearn.decomposition import PCA


# def visualize_mlp_outputs(model, model_quant, batch, block, layer='fc2'):
#     """Visualize outputs of fc1/fc2 using PCA."""
#     x, co_masks = model.get_co_embeds(batch)
#     x_quant, co_masks_quant = model_quant.get_co_embeds(batch)
    
#     with torch.no_grad():
#         # Full-precision outputs
#         x_fc1_full = model.transformer.blocks[block].mlp.fc1(x)
#         x_fc2_full = model.transformer.blocks[block].mlp.fc2(x_fc1_full)
        
#         # Quantized outputs
#         x_fc1_quant = model_quant.transformer.blocks[block].mlp.fc1(x_quant)
#         x_fc2_quant = model_quant.transformer.blocks[block].mlp.fc2(x_fc1_quant)
    
#     # Select layer to visualize
#     if layer == 'fc1':
#         full_output = x_fc1_full.cpu().numpy()
#         quant_output = x_fc1_quant.cpu().numpy()
#     else:
#         full_output = x_fc2_full.cpu().numpy()
#         quant_output = x_fc2_quant.cpu().numpy()
    
#     # Apply PCA
#     pca = PCA(n_components=2)
#     pca_full = pca.fit_transform(full_output.reshape(full_output.shape[0], -1))
#     pca_quant = pca.transform(quant_output.reshape(quant_output.shape[0], -1))
    
#     # Plot
#     plt.figure(figsize=(10, 5))
#     plt.scatter(pca_full[:, 0], pca_full[:, 1], label='Full Precision', alpha=0.5)
#     plt.scatter(pca_quant[:, 0], pca_quant[:, 1], label='Quantized', alpha=0.5)
#     plt.title(f'Block {block} MLP.{layer} Output Similarity (PCA)')
#     plt.legend()
#     plt.show()

In [15]:
from copy import deepcopy
import torch.nn.functional as F


for i in range(12):

    # Quantize MLP layers (fc1 and fc2)
    # layer_to_quantize = f"transformer.blocks.{i}.mlp"
    layer_to_quantize1 = "text_transformer.encoder.layer." + str(i) + ".intermediate"
    
    
    model_dynamic = deepcopy(model)
    torch.quantization.quantize_dynamic(
        model_dynamic, {layer_to_quantize1: quantization_config, layer_to_quantize1: quantization_config}, inplace=True
    )

    
    # # Compare weights
    # plot_weight_distributions(model, model_dynamic, block=i)
    
    # Compare activations
    results = analyze_mlp_activations(model, model_dynamic, batch=input_, block=i)
    print(f"Block {i} fc1, {results['mse_fc1']:.6f}, {results['cosine_fc1']:.6f}, {results['stats']['fc1_mean_change']:.2f}, {results['stats']['fc1_var_change']:.2f}, {results['stats']['fc1_var_weighted_change']:.6f}")

    # print(f"Block {i}:")
    # print(f"  FC1: MSE={results['mse_fc1']:.6f}, Cosine={results['cosine_fc1']:.6f}")
    # print(f"  FC2: MSE={results['mse_fc2']:.6f}, Cosine={results['cosine_fc2']:.6f}")
    # print("  Activation Stats:")
    # print(f"    FC1 Mean: Full={results['stats']['fc1_full_mean']:.6f}, Quant={results['stats']['fc1_quant_mean']:.6f}, Change={results['stats']['fc1_mean_change']:.2f}%")
    # print(f"    FC1 Var: Full={results['stats']['fc1_full_var']:.6f}, Quant={results['stats']['fc1_quant_var']:.6f}, Change={results['stats']['fc1_var_change']:.2f}%")
    # print(f"    FC1 Weighted Var Change: {results['stats']['fc1_var_weighted_change']:.6f}")
    # print(f"    FC2 Mean: Full={results['stats']['fc2_full_mean']:.6f}, Quant={results['stats']['fc2_quant_mean']:.6f}, Change={results['stats']['fc2_mean_change']:.2f}%")
    # print(f"    FC2 Var: Full={results['stats']['fc2_full_var']:.6f}, Quant={results['stats']['fc2_quant_var']:.6f}, Change={results['stats']['fc2_var_change']:.2f}%")
    # print(f"    FC2 Weighted Var Change: {results['stats']['fc2_var_weighted_change']:.6f}")
    # print("==========================================================================")



    # # Visualize outputs
    # visualize_mlp_outputs(model, model_dynamic, batch=input_, block=i, layer='fc1')
    # visualize_mlp_outputs(model, model_dynamic, batch=input_, block=i, layer='fc2')

for i in range(12):
    layer_to_quantize2 = "text_transformer.encoder.layer." + str(i) + ".output"
    model_dynamic = deepcopy(model)
    torch.quantization.quantize_dynamic(
        model_dynamic, {layer_to_quantize1: quantization_config, layer_to_quantize2: quantization_config}, inplace=True
    )

    # Compare activations
    results = analyze_mlp_activations(model, model_dynamic, batch=input_, block=i)
    print(f"Block {i} fc2, {results['mse_fc2']:.6f}, {results['cosine_fc2']:.6f}, {results['stats']['fc2_mean_change']:.2f}, {results['stats']['fc2_var_change']:.2f}, {results['stats']['fc2_var_weighted_change']:.6f}")
    

/data-4/users/mileriso/envs/.dev/lib/python3.10/site-packages/transformers/modeling_utils.py:1161: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Block 0 fc1, 0.139047, 0.067519, -288.44, -95.62, 191.245444
Block 1 fc1, 0.039054, 0.154413, 8.03, -64.82, 223.946385
Block 2 fc1, 0.040365, 0.290057, -44.90, 28.20, -106.651927
Block 3 fc1, 0.043985, 0.266469, -62.50, 1.91, -6.864872
Block 4 fc1, 0.051086, 0.221976, -68.61, 15.86, -56.574007
Block 5 fc1, 0.051729, 0.247183, -83.36, 60.03, -222.085718
Block 6 fc1, 0.041326, 0.226783, -67.10, 12.70, -47.937606
Block 7 fc1, 0.045406, 0.262018, -73.03, 11.80, -42.618263
Block 8 fc1, 0.077738, 0.639988, -72.00, -10.16, 22.231513
Block 9 fc1, 0.048419, 0.320433, -73.59, 20.74, -72.635251
Block 10 fc1, 0.029230, 0.244010, -75.99, -13.81, 56.665651
Block 11 fc1, 0.025074, 0.229656, -72.94, 44.59, -213.854694
Block 0 fc2, 0.116811, 0.859665, -6.35, 23.79, -24.368496
Block 1 fc2, 0.050451, 0.966949, 8.45, 5.62, -1.752675
Block 2 fc2, 0.038324, 0.972023, 1.31, 4.27, -1.748230
Block 3 fc2, 0.030557, 0.976623, -0.43, 2.07, -0.912058
Block 4 fc2, 0.034972, 0.978451, -0.95, 3.15, -0.732017
Block 5 

In [13]:
import torch

def get_attention_maps(model, batch):
    """Extract attention weights from all blocks."""
    attention_maps = {}
    
    # Register hooks to capture attention weights
    hooks = []
    for i in range(12):
        block = model.transformer.blocks[i].attn
        hook = block.register_forward_hook(
            lambda module, input, output, block_idx=i: attention_maps.__setitem__(f'block_{block_idx}', output[1].detach())
        )
        hooks.append(hook)
    
    # Forward pass (replace with your ViLT input format)
    with torch.no_grad():
        infer1 = model.infer( 
            batch, mask_text=False, mask_image=False, image_token_type_idx=1
        )
    
    # Remove hooks
    for hook in hooks:
        hook.remove()
    
    return attention_maps

# Get attention maps for both models
attn_full = get_attention_maps(model, input_)

In [14]:
import pandas as pd
import torch
import torch.nn.functional as F
import math

# def weighted_variance_change(full_var, quant_var):
#     """Calculate weighted variance change."""
#     percent_change = ((quant_var - full_var) / full_var) * 100
#     return percent_change * math.log(full_var)

# def analyze_mlp_activations(model, model_quant, batch, block):
#     """Compare activations of MLP layers (fc1 and fc2) for a specific block."""
#     x, co_masks = model.get_co_embeds(batch)
#     x_quant, co_masks_quant = model_quant.get_co_embeds(batch)
    
#     with torch.no_grad():
#         # Full-precision activations
#         x_fc1_full = model.transformer.blocks[block].mlp.fc1(x)
#         x_fc2_full = model.transformer.blocks[block].mlp.fc2(x_fc1_full)
        
#         # Quantized activations
#         x_fc1_quant = model_quant.transformer.blocks[block].mlp.fc1(x_quant)
#         x_fc2_quant = model_quant.transformer.blocks[block].mlp.fc2(x_fc1_quant)
    
#     # Compute metrics for fc1
#     mse_fc1 = F.mse_loss(x_fc1_full, x_fc1_quant).item()
#     cosine_fc1 = F.cosine_similarity(x_fc1_full.flatten(), x_fc1_quant.flatten(), dim=0).item()
    
#     # Compute metrics for fc2
#     mse_fc2 = F.mse_loss(x_fc2_full, x_fc2_quant).item()
#     cosine_fc2 = F.cosine_similarity(x_fc2_full.flatten(), x_fc2_quant.flatten(), dim=0).item()
    
#     # Activation statistics (mean and variance)
#     fc1_full_mean = x_fc1_full.mean().item()
#     fc1_full_var = x_fc1_full.var().item()
#     fc1_quant_mean = x_fc1_quant.mean().item()
#     fc1_quant_var = x_fc1_quant.var().item()
    
#     fc2_full_mean = x_fc2_full.mean().item()
#     fc2_full_var = x_fc2_full.var().item()
#     fc2_quant_mean = x_fc2_quant.mean().item()
#     fc2_quant_var = x_fc2_quant.var().item()
    
#     # Percent changes in means and variances
#     def percent_change(full, quant):
#         return ((quant - full) / full) * 100
    
#     fc1_mean_change = percent_change(fc1_full_mean, fc1_quant_mean)
#     fc1_var_change = percent_change(fc1_full_var, fc1_quant_var)
#     fc2_mean_change = percent_change(fc2_full_mean, fc2_quant_mean)
#     fc2_var_change = percent_change(fc2_full_var, fc2_quant_var)
    
#     # Weighted variance change
#     fc1_var_weighted_change = weighted_variance_change(fc1_full_var, fc1_quant_var)
#     fc2_var_weighted_change = weighted_variance_change(fc2_full_var, fc2_quant_var)
    
#     # Return results in a dictionary
#     return {
#         'fc1': {
#             'mse': mse_fc1,
#             'cosine': cosine_fc1,
#             'mean_change': fc1_mean_change,
#             'var_change': fc1_var_change,
#             'var_weighted_change': fc1_var_weighted_change,
#         },
#         'fc2': {
#             'mse': mse_fc2,
#             'cosine': cosine_fc2,
#             'mean_change': fc2_mean_change,
#             'var_change': fc2_var_change,
#             'var_weighted_change': fc2_var_weighted_change,
#         }
#     }

def analyze_attention_activations(model, model_quant, batch, block):
    """Compare activations of attention sub-modules for a specific block."""
    # Get attention maps for full-precision and quantized models
    attn_full = get_attention_maps(model, batch)
    attn_quant = get_attention_maps(model_quant, batch)
    
    # Extract attention outputs for the specified block
    attn_full_block = attn_full[f'block_{block}']
    attn_quant_block = attn_quant[f'block_{block}']
    
    # Compute MSE and Cosine Similarity
    mse = F.mse_loss(attn_full_block, attn_quant_block).item()
    cosine = F.cosine_similarity(attn_full_block.flatten(), attn_quant_block.flatten(), dim=0).item()
    
    # Compute mean and variance
    full_mean = attn_full_block.mean().item()
    full_var = attn_full_block.var().item()
    quant_mean = attn_quant_block.mean().item()
    quant_var = attn_quant_block.var().item()
    
    # Percent changes in mean and variance
    def percent_change(full, quant):
        return ((quant - full) / full) * 100
    
    mean_change = percent_change(full_mean, quant_mean)
    var_change = percent_change(full_var, quant_var)
    
    # Weighted variance change
    var_weighted_change = weighted_variance_change(full_var, quant_var)
    
    # Return results in a dictionary
    return {
        'mse': mse,
        'cosine': cosine,
        'mean_change': mean_change,
        'var_change': var_change,
        'var_weighted_change': var_weighted_change,
    }

# Initialize an empty DataFrame
columns = ['Layer', 'MSE', 'Cosine', 'Mean (%)', 'Var (%)', 'Weighted Var']
results_df = pd.DataFrame(columns=columns)

# Analyze each block
for i in range(12):
    print(f"Block {i}")

    # Quantize the attention block
    layer_to_quantize = f"transformer.blocks.{i}.attn"
    model_dynamic = deepcopy(model)
    torch.quantization.quantize_dynamic(
        model_dynamic, {layer_to_quantize: quantization_config}, inplace=True
    )
    
    # Analyze attention outputs
    attn_results = analyze_attention_activations(model, model_dynamic, batch=input_, block=i)
    attn_row = pd.DataFrame({
        'Layer': [f"{i} attn"],
        'MSE': [attn_results['mse']],
        'Cosine': [attn_results['cosine']],
        'Mean (%)': [attn_results['mean_change']],
        'Var (%)': [attn_results['var_change']],
        'Weighted Var': [attn_results['var_weighted_change']],
    })
    results_df = pd.concat([results_df, attn_row], ignore_index=True)

    # Quantize MLP layers (fc1 and fc2)
    layer_to_quantize = f"transformer.blocks.{i}.mlp"
    model_dynamic = deepcopy(model)
    torch.quantization.quantize_dynamic(
        model_dynamic, {layer_to_quantize: quantization_config}, inplace=True
    )

    
    # Analyze MLP outputs
    mlp_results = analyze_mlp_activations(model, model_dynamic, batch=input_, block=i)
    fc1_row = pd.DataFrame({
        'Layer': [f"{i} fc1"],
        'MSE': [mlp_results['mse_fc1']],
        'Cosine': [mlp_results['cosine_fc1']],
        'Mean (%)': [mlp_results['stats']['fc1_mean_change']],
        'Var (%)': [mlp_results['stats']['fc1_var_change']],
        'Weighted Var': [mlp_results['stats']['fc1_var_weighted_change']],
    })
    results_df = pd.concat([results_df, fc1_row], ignore_index=True)
    
    fc2_row = pd.DataFrame({
        'Layer': [f"{i} fc2"],
        'MSE': [mlp_results['mse_fc2']],
        'Cosine': [mlp_results['cosine_fc2']],
        'Mean (%)': [mlp_results['stats']['fc2_mean_change']],
        'Var (%)': [mlp_results['stats']['fc2_var_change']],
        'Weighted Var': [mlp_results['stats']['fc2_var_weighted_change']],
    })
    results_df = pd.concat([results_df, fc2_row], ignore_index=True)

# Save the results to a CSV file
results_df.to_csv('quantization_results.csv', index=False)

# Print the results
print(results_df)

Block 0


/tmp/ipykernel_69740/2502827313.py:139: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, attn_row], ignore_index=True)


Block 1
Block 2
Block 3
Block 4
Block 5
Block 6
Block 7
Block 8
Block 9
Block 10
Block 11
      Layer          MSE    Cosine    Mean (%)    Var (%)  Weighted Var
0    0 attn     0.000165  0.938830    0.000006   2.176645    -14.453415
1     0 fc1    11.346266  0.880878  -17.475320  -7.426643    -28.702832
2     0 fc2  2157.084229  0.538112  -26.656825 -81.284223   -650.490038
3    1 attn     0.000194  0.905887    0.000000   1.051419     -7.254471
4     1 fc1     8.785393  0.875470  -18.713199  -0.345244     -1.202759
5     1 fc2   744.901123  0.374956 -166.489665 -90.500806   -611.745766
6    2 attn     0.000248  0.855842    0.000000   8.616831    -61.385724
7     2 fc1     4.771626  0.900590   -2.276569   2.643329      8.112577
8     2 fc2   438.183472  0.230853  -32.945192 -94.845369   -582.099789
9    3 attn     0.000126  0.866064    0.000000  11.308649    -87.736169
10    3 fc1     4.602452  0.868584    0.129894   5.296170     14.472023
11    3 fc2   126.995857  0.822448  143.814773

# Attention Analysis

In [60]:
import torch

def get_attention_maps(model, batch):
    """Extract attention weights from all blocks."""
    attention_maps = {}
    
    # Register hooks to capture attention weights
    hooks = []
    for i in range(12):
        block = model.transformer.blocks[i].attn
        hook = block.register_forward_hook(
            lambda module, input, output, block_idx=i: attention_maps.__setitem__(f'block_{block_idx}', output[1].detach())
        )
        hooks.append(hook)
    
    # Forward pass (replace with your ViLT input format)
    with torch.no_grad():
        infer1 = model.infer( 
            batch, mask_text=False, mask_image=False, image_token_type_idx=1
        )
    
    # Remove hooks
    for hook in hooks:
        hook.remove()
    
    return attention_maps


In [61]:
import torch
import torch.nn.functional as F
import math

def weighted_variance_change(full_var, quant_var):
    """Calculate weighted variance change."""
    percent_change = ((quant_var - full_var) / full_var) * 100
    return percent_change * math.log(full_var)

def analyze_attention_outputs(model, model_quant, batch):
    """Compare attention outputs for all blocks."""
    # Get attention maps for full-precision and quantized models
    attn_full = get_attention_maps(model, batch)
    attn_quant = get_attention_maps(model_quant, batch)
    
    results = {}
    for i in range(12):
        # Extract attention outputs for block i
        attn_full_block = attn_full[f'block_{i}']
        attn_quant_block = attn_quant[f'block_{i}']
        
        # Compute MSE and Cosine Similarity
        mse = F.mse_loss(attn_full_block, attn_quant_block).item()
        cosine = F.cosine_similarity(attn_full_block.flatten(), attn_quant_block.flatten(), dim=0).item()
        
        # Compute mean and variance
        full_mean = attn_full_block.mean().item()
        full_var = attn_full_block.var().item()
        quant_mean = attn_quant_block.mean().item()
        quant_var = attn_quant_block.var().item()
        
        # Percent changes in mean and variance
        def percent_change(full, quant):
            return ((quant - full) / full) * 100
        
        mean_change = percent_change(full_mean, quant_mean)
        var_change = percent_change(full_var, quant_var)
        
        # Weighted variance change
        var_weighted_change = weighted_variance_change(full_var, quant_var)
        
        # Store results for block i
        results[f'block_{i}'] = {
            'mse': mse,
            'cosine': cosine,
            'full_mean': full_mean,
            'full_var': full_var,
            'quant_mean': quant_mean,
            'quant_var': quant_var,
            'mean_change': mean_change,
            'var_change': var_change,
            'var_weighted_change': var_weighted_change,
        }
    
    return results

# Example usage
results = analyze_attention_outputs(model, model_dynamic, batch=input_)

# Print results
for i in range(12):
    block_results = results[f'block_{i}']
    print(f"Block {i} Attention Metrics:")
    print(f"  MSE: {block_results['mse']:.6f}")
    print(f"  Cosine Similarity: {block_results['cosine']:.6f}")
    print(f"  Mean: Full={block_results['full_mean']:.6f}, Quant={block_results['quant_mean']:.6f}, Change={block_results['mean_change']:.2f}%")
    print(f"  Variance: Full={block_results['full_var']:.6f}, Quant={block_results['quant_var']:.6f}, Change={block_results['var_change']:.2f}%")
    print(f"  Weighted Variance Change: {block_results['var_weighted_change']:.6f}")

Block 0 Attention Metrics:
  MSE: 0.000098
  Cosine Similarity: 0.965466
  Mean: Full=0.003717, Quant=0.003717, Change=0.00%
  Variance: Full=0.001314, Quant=0.001405, Change=6.90%
  Weighted Variance Change: -45.754820
Block 1 Attention Metrics:
  MSE: 0.000189
  Cosine Similarity: 0.908517
  Mean: Full=0.003717, Quant=0.003717, Change=0.00%
  Variance: Full=0.000969, Quant=0.001045, Change=7.84%
  Weighted Variance Change: -54.400835
Block 2 Attention Metrics:
  MSE: 0.000237
  Cosine Similarity: 0.861100
  Mean: Full=0.003717, Quant=0.003717, Change=0.00%
  Variance: Full=0.000803, Quant=0.000862, Change=7.43%
  Weighted Variance Change: -52.929109
Block 3 Attention Metrics:
  MSE: 0.000112
  Cosine Similarity: 0.879773
  Mean: Full=0.003717, Quant=0.003717, Change=0.00%
  Variance: Full=0.000426, Quant=0.000467, Change=9.62%
  Weighted Variance Change: -74.649181
Block 4 Attention Metrics:
  MSE: 0.000086
  Cosine Similarity: 0.877802
  Mean: Full=0.003717, Quant=0.003717, Change=0